In [18]:
# over welke periode gaat het dataframe: 10 jaar
# aggregratie op jaar niveau, eventueel later op week niveau

# welke X-en:
# PC4
# LON 
# LAT
# Res Regio
# Gemeente
# tem: hoe warm was het jaar?

# sjv_totaal
# factor_maand
# factor
# week_verbruik_kw
# laadpalen
# electrische auto's
# warmtepompen
# Opwekking zonneenergie
# Opwekking Windenergie
# Decentrale opwekgegevens
# Knelpunt indicatie (Provincie?)

In [143]:
import pandas as pd
import numpy as np
import os
import glob
import zipfile
import seaborn as sns
!pip install altair
import altair as alt

  Using cached altair-4.1.0-py3-none-any.whl (727 kB)
  Using cached toolz-0.11.1-py3-none-any.whl (55 kB)


In [144]:
# show all columns in the dataframe
pd.set_option('max_columns', None)

In [145]:
# variables used in script
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

# Alle benodige ht5 dataframes inlezen

In [146]:
# decentrale_opwekkingsgegevens gegevens inlezen
df_opwek = pd.read_hdf('decentrale_opwekkingsgegevens_data.h5')
# kleinverbruikgegevens gegevens inlezen
df_verbruik = pd.read_hdf('kleinverbruikgegevens_data.h5')
# mapping van CBS buurt naar PC4
df_cbs_buurt_pc4 = pd.read_hdf('cbs_buurt_pc4.h5')
# mapping van PC4 buurt naar RES regio
df_pc4_res = pd.read_hdf('pc4_res.h5')
# cbs data
df_cbs = pd.read_hdf('cbs_data.h5')

# Nieuw master dataframe maken

In [147]:
# maak nieuw dataframe met de jaren
df = pd.DataFrame(pd.date_range(start='1/1/2010', end='31/12/2020' , freq='Y'), columns=['DATUM']).sort_values(by='DATUM', ascending=False)
df['JAAR'] = df['DATUM'].dt.year

# Vebruik toevoegen aan dataframe

In [148]:
# df_verbruik is nog per pc6, dus eerst omzetten naar pc4
df_verbruik_pc4 = df_verbruik.groupby(['PC4','JAAR']
                                      , as_index=False
                                     ).agg({'SJV_TOTAAL': 'sum'
                                            ,'AANSLUITINGEN_AANTAL': 'sum'  
                                            ,'LEVERINGSRICHTING_PERC': 'mean'
                                           }
                                          )

# join df met df_verbruik
df = pd.merge(df, df_verbruik_pc4, on='JAAR', how='left')

# Opwek toevoegen aan dataframe

In [149]:
# jaar toevoegen
# 2020-07-01 verwijderen, de maand van de peildatum moet januari zijn
df_opwek = df_opwek[df_opwek['Peildatum'].dt.month == 1]
# 2020-01-01 wordt 2019, 2021-01-01 wordt 2020
df_opwek['JAAR'] = df_opwek['Peildatum'].dt.year - 1
# PC4 toevoegen
df_cbs_buurt_pc4 = df_cbs_buurt_pc4.rename(columns={'Buurt2020': 'CBS Buurtcode'})
df_opwek = pd.merge(df_opwek, df_cbs_buurt_pc4, on='CBS Buurtcode', how='left', suffixes=('','_2'))
# df_opwek group by PC4 en JAAR
df_opwek_pc4 = df_opwek.groupby(['PC4','JAAR'], as_index=False).agg({'Gemeente':'max'
                                                                     , 'Aantal aansluitingen in CBS-buurt': 'sum'
                                                                     , 'Aantal aansluitingen met opwekinstallatie':'sum'
                                                                     , 'Opgesteld vermogen':'sum'
                                                                    }
                                                                   )
# rename columns
df_opwek_pc4 = df_opwek_pc4.rename(columns={'Aantal aansluitingen in CBS-buurt': 'AANTAL AANSLUITINGEN IN PC4'
                                            , 'Gemeente': 'GEMEENTE'
                                            , 'Aantal aansluitingen met opwekinstallatie': 'AANTAL AANSLUITINGEN MET OPWEKINSTALLATIE'
                                            , 'Opgesteld vermogen': 'OPGESTELD VERMOGEN'
                                           }
                                  )

# join df met df_opwek_pc4
df = pd.merge(df, df_opwek_pc4, on=['JAAR','PC4'], how='left')

# RES regio toevoegen aan dataframe

In [150]:
df = pd.merge(df, df_pc4_res, on=['PC4'], how='left')

In [151]:
#df.info()

# CBS data voor 2015-2019 toevoegen. Selectie van relevante features nog niet  gedaan¶

In [152]:
df_cbs['PC4'].head()

0    1011
1    1012
2    1013
3    1014
4    1015
Name: PC4, dtype: int64

In [153]:
df_cbs.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
PC4,20308.0,5669.206421,2671.038412,1011.0,3328.00,5702.00,8043.000,9999.0
INWONER,20308.0,3619.416831,9000.093396,-99997.0,660.00,2650.00,6910.000,28410.0
MAN,20308.0,1421.189679,8552.518942,-99997.0,340.00,1320.00,3395.000,14195.0
VROUW,20308.0,1371.024769,9057.870561,-99997.0,320.00,1315.00,3490.000,14215.0
INW_014,20308.0,-1821.440467,15758.688512,-99997.0,105.00,410.00,1090.000,6535.0
INW_1524,20308.0,-2122.197016,16125.515560,-99997.0,75.00,300.00,795.000,4905.0
INW_2544,20308.0,-687.121430,13224.500799,-99997.0,130.00,565.00,1650.000,9710.0
INW_4564,20308.0,218.849616,9960.439015,-99997.0,215.00,780.00,1905.000,8860.0
INW_65PL,20308.0,-1181.677221,13993.603006,-99997.0,120.00,485.00,1255.000,5310.0
GEBOORTE,20308.0,-23239.402501,42285.093479,-99997.0,5.00,25.00,65.000,430.0


In [154]:
# Merge cbs data for years 2015-2019. Other years to be loacted or constructed 
df = pd.merge(df, df_cbs, on=['JAAR','PC4'], how='left')

In [155]:
# Review results
pd.options.display.max_rows = 500
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
JAAR,17130.0,2015.010274,3.168513,2010.000,2012.000000,2015.00000,2018.000000,2020.0
PC4,17130.0,7175.969527,1770.313330,4251.000,5563.000000,7429.00000,8917.000000,9999.0
SJV_TOTAAL,17130.0,7419.743900,7319.597724,26.000,1617.250000,4894.50000,11216.750000,52390.0
AANSLUITINGEN_AANTAL,17130.0,1609.624635,1679.368256,10.000,259.000000,902.50000,2607.000000,10063.0
LEVERINGSRICHTING_PERC,17130.0,93.662165,7.712743,26.895,90.444444,96.42182,99.693618,100.0
AANTAL AANSLUITINGEN IN PC4,2435.0,1867.102259,1774.069860,17.000,418.000000,1213.00000,2914.500000,8897.0
AANTAL AANSLUITINGEN MET OPWEKINSTALLATIE,2435.0,304.359754,301.218239,10.000,81.000000,205.00000,438.000000,2475.0
OPGESTELD VERMOGEN,2435.0,1330.798768,1217.099305,33.000,447.000000,955.00000,1848.000000,8278.0
INWONER,7731.0,3599.881387,4707.024553,-99997.000,600.000000,2185.00000,6017.500000,23005.0
MAN,7731.0,1654.812702,4706.014075,-99997.000,315.000000,1100.00000,2965.000000,11495.0


In [184]:
df[df['JAAR']==2019].head(20).transpose()

,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597
DATUM,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00,2019-12-31 00:00:00
JAAR,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019
PC4,4251,4254,4255,4261,4264,4265,4266,4267,4268,4269,4271,4273,4281,4283,4284,4285,4286,4287,4288,4611
SJV_TOTAAL,22391.0,10609.0,6383.0,12843.0,7366.0,3975.0,1740.0,1226.0,1643.0,1318.0,6024.0,7396.0,4804.0,4130.0,3067.0,8456.0,7359.0,457.0,849.0,24335.0
AANSLUITINGEN_AANTAL,4720,2446,1630,2495,1284,687,331,160,302,161,1107,1639,1022,747,753,2101,1607,87,145,5427
LEVERINGSRICHTING_PERC,93.148667,86.7605,84.559487,92.70069,95.191429,91.291562,91.131667,90.90625,90.272667,89.634444,90.889138,87.982619,92.762456,90.974286,88.45825,91.496796,83.731724,88.96,88.915556,94.450549
GEMEENTE,Altena,NaN,NaN,Altena,Altena,Altena,Altena,NaN,Altena,NaN,NaN,NaN,Altena,Altena,NaN,Altena,NaN,Altena,Altena,Bergen op Zoom
AANTAL AANSLUITINGEN IN PC4,1922.0,NaN,NaN,289.0,1179.0,615.0,239.0,NaN,223.0,NaN,NaN,NaN,857.0,522.0,NaN,165.0,NaN,116.0,108.0,5265.0
AANTAL AANSLUITINGEN MET OPWEKINSTALLATIE,117.0,NaN,NaN,39.0,76.0,79.0,34.0,NaN,25.0,NaN,NaN,NaN,89.0,63.0,NaN,39.0,NaN,19.0,15.0,261.0
OPGESTELD VERMOGEN,335.0,NaN,NaN,197.0,383.0,458.0,119.0,NaN,97.0,NaN,NaN,NaN,433.0,227.0,NaN,203.0,NaN,83.0,54.0,569.0


# Save dataframe 

In [157]:
# data locatie om de ht5 file op te slaan
data_processed_location = '../processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

In [158]:
try:
    os.remove('master_data.h5')
except:
    print ('File nog in gebruik of niet gevonden')

store = pd.HDFStore('master_data.h5')
store['master_data'] = df
store.close()

C:\Users\jcdek\anaconda3\envs\EB\lib\site-packages\IPython\core\interactiveshell.py:3427: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['GEMEENTE', 'RES', 'M_INKHH'], dtype='object')]

  exec(code_obj, self.user_global_ns, self.user_ns)


# QC plots

In [205]:
df15_19 = df.loc[(df['JAAR'] > 2014) & (df['JAAR'] < 2020)]

In [226]:
df15_19 = df15_19.replace(-99997, np.nan)
df15_19 = df15_19.fillna(method='ffill')

In [259]:
#
# Group by PC4
#    syntax: df.groupby(['att1', 'att2']).agg({'att1': "count", 'att3': "sum",'att4': 'mean'})

df15_19_PC4 = df15_19.groupby(['JAAR','PC4']).agg({'SJV_TOTAAL' : 'sum', 'INWONER' : 'sum', 
                                'AANSLUITINGEN_AANTAL' : 'sum'})
df15_19_PC4 = df15_19_PC4.reset_index()

#
# Group by RES
#    syntax: df.groupby(['att1', 'att2']).agg({'att1': "count", 'att3': "sum",'att4': 'mean'})

df15_19_RES = df15_19.groupby(['JAAR','RES']).agg({'SJV_TOTAAL' : 'sum', 'INWONER' : 'sum', 
                                'AANSLUITINGEN_AANTAL' : 'sum'})
df15_19_RES = df15_19_RES.reset_index()


In [250]:
alt.data_transformers.disable_max_rows()

select_year = alt.selection_single(
    name='select', fields=['JAAR'], init={'JAAR': 2015},
    bind=alt.binding_range(min=2015, max=2019, step=1))

alt.Chart(df15_19_PC4).mark_point(filled=True).encode(
    alt.X('INWONER'),
    alt.Y('SJV_TOTAAL'),
    alt.Color('RES:N')).add_selection(select_year).transform_filter(select_year).properties(
    width=600,height=600)


alt.Chart(...)

In [276]:
verbruik = alt.Chart(df15_19_RES).mark_line().encode(
                     x='JAAR',
                     y='SJV_TOTAAL',
                     color='RES:N',
                     strokeDash='RES:N',).properties(width=350,height=350)
inwoners = alt.Chart(df15_19_RES).mark_line().encode(
                     x='JAAR',
                     y='INWONER',
                     color='RES:N',
                     strokeDash='RES:N',).properties(width=350,height=350)
verbruik | inwoners

alt.HConcatChart(...)

In [282]:
#adjust limits and clip data

verbruik = alt.Chart(df15_19_RES).mark_line(clip=True).encode(
                     alt.X('JAAR'),
                     alt.Y('SJV_TOTAAL',scale=alt.Scale(domain=(800000, 1600000))),
                     color='RES:N',
                     strokeDash='RES:N',).properties(width=350,height=350)
    
inwoners = alt.Chart(df15_19_RES).mark_line(clip=True).encode(
                     alt.X('JAAR'),
                     alt.Y('INWONER',scale=alt.Scale(domain=(400000, 800000))),
                     color='RES:N',
                     strokeDash='RES:N',).properties(width=350,height=350)
verbruik | inwoners


alt.HConcatChart(...)